In [1]:
# @itsvenu_
# AutoEncoders

# data preprocessing steps are same 
# as in 
# https://github.com/itsvenu/MachineLearning-A2Z/blob/main/models/32_BoltzmannMachines/31_BoltzmannMachines.ipynb

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
movies = pd.read_csv('../../data-dl/ml-1m/movies.dat', 
                      sep = "::", header=None, engine='python',
                     encoding = 'latin-1')
movies.head()
# movie-name # genre

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users = pd.read_csv('../../data-dl/ml-1m/users.dat', sep = "::", header=None, engine='python', encoding = 'latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings = pd.read_csv('../../data-dl/ml-1m/ratings.dat', sep = "::", header=None, engine='python', encoding = 'latin-1')
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# change training data set from Boltzmann Machine
training_set = pd.read_csv('../../data-dl/ml-100k/u2.base', delimiter = '\t', header=None)
training_set.head()

,0,1,2,3
0,1,3,4,878542960
1,1,4,3,876893119
2,1,5,3,889751712
3,1,6,5,887431973
4,1,7,4,875071561


In [7]:
training_set = np.array(training_set, dtype='int')
type(training_set)

numpy.ndarray

In [8]:
test_set = pd.read_csv('../../data-dl/ml-100k/u2.test', delimiter = '\t', header=None)
test_set = np.array(test_set, dtype='int')
test_set[:5]

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         8,         1, 875072484],
       [        1,         9,         5, 878543541],
       [        1,        21,         1, 878542772]])

In [9]:
# get number of movies and users
# to prepare a matrix of rows-users; cols-movies; values-ratingsl
# if no rating from a user-i for movie-i, put 0

nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0]))) #max/total number of users
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1]))) # max/total number of movies

print(nb_users)
print(nb_movies)

943
1682


In [10]:
# list of lists
def convert(dat):
    
    new_dat = []
    for id_users in range(1, nb_users+1):
        id_movies = dat[:, 1][dat[:,0] == id_users]
        id_ratings = dat[:, 2][dat[:,0] == id_users]
        i_ratings = np.zeros(nb_movies)
        i_ratings[id_movies-1] = id_ratings
        new_dat.append(list(i_ratings))
    
    return(new_dat)

In [11]:
training_set = convert(training_set)
test_set = convert(test_set)

In [15]:
# make torch-tensors from the training/test dat

training_set = torch.FloatTensor(training_set)
training_set

tensor([[0., 0., 4.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [16]:
test_set = torch.FloatTensor(test_set)
test_set

tensor([[5., 3., 0.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [19]:
# architecture
# SAE - stacked auto encoders
# fc - full-connection

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # first hidden layer
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        
        return(x)

In [21]:

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [23]:
# train

nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
            
    print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.6457)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0542)
epoch: 4loss: tensor(1.0393)
epoch: 5loss: tensor(1.0315)
epoch: 6loss: tensor(1.0268)
epoch: 7loss: tensor(1.0239)
epoch: 8loss: tensor(1.0224)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0198)
epoch: 11loss: tensor(1.0186)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0175)
epoch: 14loss: tensor(1.0175)
epoch: 15loss: tensor(1.0167)
epoch: 16loss: tensor(1.0168)
epoch: 17loss: tensor(1.0164)
epoch: 18loss: tensor(1.0165)
epoch: 19loss: tensor(1.0159)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0157)
epoch: 22loss: tensor(1.0159)
epoch: 23loss: tensor(1.0151)
epoch: 24loss: tensor(1.0156)
epoch: 25loss: tensor(1.0145)
epoch: 26loss: tensor(1.0130)
epoch: 27loss: tensor(1.0117)
epoch: 28loss: tensor(1.0100)
epoch: 29loss: tensor(1.0093)
epoch: 30loss: tensor(1.0057)
epoch: 31loss: tensor(1.0039)
epoch: 32loss: tensor(1.0000)
epoch: 33loss: tensor(0.9988)
epoch: 34loss: tens

In [25]:
# testing

test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9533)
